# Getting Comfortable with Latitude and Longitude

In the next few lectures, we will be working with **geospatial data**---that is, data involving locations on Earth.

Locations on Earth are specified using **latitude** and **longitude**.

- Longitude specifies east-west position.
- Latitude specifies north-south position.

For example, the location below is at a longitude of $\lambda = -45^\circ$ and a latitude of $\phi = 30^\circ$.

<img src="data/9.1-figure1.png" width="500">

The Earth is split into:

- Northern and Southern Hemispheres by the **equator** (at $0^\circ$ latitude)
- Eastern and Western Hemispheres by the **prime meridian** (at $0^\circ$ longitude).

Latitude and longitude represent angles relative to the equator and prime meridian, respectively.

<img src="data/9.1-figure2.png" width="500">

Note that longitudes west of the prime meridian and latitudes south of the equator are negative.

## World Cities Data

We will work with a dataset containing the locations and populations of world "cities". (As you will see, some of the places in this dataset are not really cities.)

In [2]:
import pandas as pd
df_cities = pd.read_csv("data/worldcities.csv")
df_cities

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
0,Qal eh-ye Now,Qal eh-ye,34.983000,63.133300,2997.0,Afghanistan,AF,AFG,Badghis
1,Chaghcharan,Chaghcharan,34.516701,65.250001,15000.0,Afghanistan,AF,AFG,Ghor
2,Lashkar Gah,Lashkar Gah,31.582998,64.360000,201546.0,Afghanistan,AF,AFG,Hilmand
3,Zaranj,Zaranj,31.112001,61.886998,49851.0,Afghanistan,AF,AFG,Nimroz
4,Tarin Kowt,Tarin Kowt,32.633298,65.866699,10000.0,Afghanistan,AF,AFG,Uruzgan
...,...,...,...,...,...,...,...,...,...
7317,Mutare,Mutare,-18.970019,32.650038,216785.0,Zimbabwe,ZW,ZWE,Manicaland
7318,Kadoma,Kadoma,-18.330006,29.909947,56400.0,Zimbabwe,ZW,ZWE,Mashonaland West
7319,Chitungwiza,Chitungwiza,-18.000001,31.100003,331071.0,Zimbabwe,ZW,ZWE,Harare
7320,Harare,Harare,-17.817790,31.044709,1557406.5,Zimbabwe,ZW,ZWE,Harare


### Exercise 1

What are the northernmost and southernmost "cities" in the world?

(You may want to try restricting to places with a population of at least 1000.)

In [5]:
df_cities_pop_over_1k = df_cities[df_cities["pop"] > 1000]
df_cities_pop_over_1k

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
0,Qal eh-ye Now,Qal eh-ye,34.983000,63.133300,2997.0,Afghanistan,AF,AFG,Badghis
1,Chaghcharan,Chaghcharan,34.516701,65.250001,15000.0,Afghanistan,AF,AFG,Ghor
2,Lashkar Gah,Lashkar Gah,31.582998,64.360000,201546.0,Afghanistan,AF,AFG,Hilmand
3,Zaranj,Zaranj,31.112001,61.886998,49851.0,Afghanistan,AF,AFG,Nimroz
4,Tarin Kowt,Tarin Kowt,32.633298,65.866699,10000.0,Afghanistan,AF,AFG,Uruzgan
...,...,...,...,...,...,...,...,...,...
7317,Mutare,Mutare,-18.970019,32.650038,216785.0,Zimbabwe,ZW,ZWE,Manicaland
7318,Kadoma,Kadoma,-18.330006,29.909947,56400.0,Zimbabwe,ZW,ZWE,Mashonaland West
7319,Chitungwiza,Chitungwiza,-18.000001,31.100003,331071.0,Zimbabwe,ZW,ZWE,Harare
7320,Harare,Harare,-17.817790,31.044709,1557406.5,Zimbabwe,ZW,ZWE,Harare


In [10]:
# find the northmost city with a population over 1,000
df_cities_pop_over_1k[df_cities_pop_over_1k["lat"] == df_cities_pop_over_1k["lat"].max()]

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
5732,Longyearbyen,Longyearbyen,78.216684,15.549996,1232.0,Svalbard and Jan Mayen Islands,SJ,SJM,Svalbard


In [7]:
# find the southmost city with a population over 1,000
df_cities_pop_over_1k[df_cities_pop_over_1k["lat"] == df_cities_pop_over_1k["lat"].min()]

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
1641,Puerto Williams,Puerto Williams,-54.933302,-67.61671,2381.0,Chile,CL,CHL,Magallanes y Antártica Chilena


### Exercise 2

Let's call a place a "city" if it has a population of at least 30,000. What are the westernmost and easternmost cities in the world?

Look these cities up on a map. How far apart are these cities really?

In [11]:
df_cities_pop_over_30k = df_cities[df_cities["pop"] > 30000]
df_cities_pop_over_30k

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
2,Lashkar Gah,Lashkar Gah,31.582998,64.360000,201546.0,Afghanistan,AF,AFG,Hilmand
3,Zaranj,Zaranj,31.112001,61.886998,49851.0,Afghanistan,AF,AFG,Nimroz
6,Asadabad,Asadabad,34.866000,71.150005,48400.0,Afghanistan,AF,AFG,Kunar
7,Taloqan,Taloqan,36.729999,69.540004,64256.0,Afghanistan,AF,AFG,Takhar
12,Mayda Shahr,Mayda Shahr,34.450002,68.799997,35008.0,Afghanistan,AF,AFG,Wardak
...,...,...,...,...,...,...,...,...,...
7317,Mutare,Mutare,-18.970019,32.650038,216785.0,Zimbabwe,ZW,ZWE,Manicaland
7318,Kadoma,Kadoma,-18.330006,29.909947,56400.0,Zimbabwe,ZW,ZWE,Mashonaland West
7319,Chitungwiza,Chitungwiza,-18.000001,31.100003,331071.0,Zimbabwe,ZW,ZWE,Harare
7320,Harare,Harare,-17.817790,31.044709,1557406.5,Zimbabwe,ZW,ZWE,Harare


In [12]:
# find the eastmost city with a population over 30,000
df_cities_pop_over_30k[df_cities_pop_over_30k["lng"] == df_cities_pop_over_30k["lng"].max()]

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
2519,Suva,Suva,-18.133016,178.441707,131835.0,Fiji,FJ,FJI,Central


In [13]:
# find the westmost city with a population over 30,000
df_cities_pop_over_30k[df_cities_pop_over_30k["lng"] == df_cities_pop_over_30k["lng"].min()]

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
6012,Nukualofa,Nukualofa,-21.138512,-175.220564,33139.0,Tonga,TO,TON,NaN


#### Distance between Suva and Nuku'alofa

Air distance: 463 miles / 744km

Ref: [Arund the World 360](https://www.aroundtheworld360.com/distance/suva_fj/nukualofa_to/)

### Exercise 3

Which state in the U.S. is closest to Morocco? (Take a guess before you do this exercise.)

You can answer this question by finding the closest cities in the United States to Casablanca, Morocco (made famous by the 1942 movie).

In [26]:
# find the closest US city to the Casablanca, Morocco
from sklearn.metrics import pairwise_distances

df_usa_cities = df_cities[df_cities["iso3"] == "USA"]
df_casablanca = df_cities[df_cities["city"] == "Casablanca"].iloc[0]
df_usa_cities["distance_to_casablanca"] = pairwise_distances(
    df_usa_cities[["lat", "lng"]],
    df_casablanca[["lat", "lng"]].values.reshape(1, -1)
)

df_usa_cities.sort_values("distance_to_casablanca").head(1)

/tmp/ipykernel_6440/4289671861.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_usa_cities["distance_to_casablanca"] = pairwise_distances(


,city,city_ascii,lat,lng,pop,country,iso2,iso3,province,distance_to_casablanca
6485,Calais,Calais,45.165989,-67.242392,1781.5,United States of America,US,USA,Maine,60.73743


## Problems with Euclidean Distance

In Exercise 3, you used Euclidean distance to measure the distance between two points $\vec x_1 = (\lambda_1, \phi_1)$ and $\vec x_2 = (\lambda_2, \phi_2)$. That is, you calculated the distance as:

$$ d(\vec x_1, \vec x_2) = \sqrt{(\lambda_1 - \lambda_2)^2 + (\phi_1 - \phi_2)^2}. $$

However, there are problems with Euclidean distance, as we now explore.

Which city is closer to London: Vancouver or Singapore? If we use Euclidean distance:

In [38]:
from sklearn.metrics import pairwise_distances

df_cities_indexed = df_cities.set_index(["city", "country"])
cities = [("London", "United Kingdom"),
          ("Vancouver", "Canada"),
          ("Singapore", "Singapore")]

pairwise_distances(df_cities_indexed.loc[cities, ["lat", "lng"]])

array([[  0.        , 123.02507295, 115.46007346],
       [123.02507295,   0.        , 231.99329028],
       [115.46007346, 231.99329028,   0.        ]])

Singapore appears to be closer than Vancouver (distance of 115 vs. 123). But is it really? A flight between London and Vancouver takes 9-10 hours; a flight between London and Singapore takes 13-15 hours.

Because the Earth is (approximately) a sphere, the shortest path between two locations may not be a straight line, which is what Euclidean distance measures.

Instead, the shortest path is along the **great circle** passing through the two points. A great circle is any line going around the Earth that divides the sphere into two equal hemispheres.

<img src="https://camo.githubusercontent.com/445680e66ecafe2c93e04c723bdd8ee2e7c9189bb78ae90e78d5f74b66d4d642/68747470733a2f2f6769746875622e636f6d2f646c73756e2f706f64732f626c6f622f6d61737465722f31322d47656f7370617469616c2d446174612f696d672f686176657273696e652e706e673f7261773d31" />

To appreciate the difference between the shortest path and the straight line, take a look at [this app](https://academo.org/demos/geodesics/). This phenomenon is likely familiar to you if you have taken a transcontinental flight.

To calculate the length of the shortest path, we need a new distance metric.

**Haversine distance** calculates the distance between two points on a sphere. It is defined as:
$$ d({\bf x}_1, {\bf x}_2) = 2r \arcsin\left( \sqrt{\sin^2\left( \frac{\phi_1 - \phi_2}{2} \right) + \cos(\phi_1) \cos(\phi_2) \sin^2\left( \frac{\lambda_1 - \lambda_2}{2} \right)} \right),$$
where $r$ is the radius of the sphere. Note that the latitudes $\phi_j$ and longitudes $\lambda_j$ must be in _radians_.

Because the Earth is not actually a sphere, Haversine distance is only approximate for measuring distance between two locations on Earth. But it is much better than Euclidean distance.

We can calculate Haversine distance using Scikit-Learn by specifying `metric="haversine"`. Don't forget to convert latitude and longitude, which are in degrees, to radians by multiplying by $\pi / 180$.

In [40]:
# convert latitude and longitude to radians
import numpy as np
radians = np.pi / 180 * df_cities_indexed.loc[cities, ["lat", "lng"]]

pairwise_distances(radians, metric="haversine")

array([[0.        , 1.18980004, 1.70380015],
       [1.18980004, 0.        , 2.01301207],
       [1.70380015, 2.01301207, 0.        ]])

Now, Vancouver is correctly identified as being closer to London (distance of 1.19) than Singapore (distance of 1.70).

Note that Scikit-Learn's Haversine distance assumes that the radius is $r=1$. If you want the distance in miles (kilometers), you have to multiply by the radius of the Earth, which is 3,960 miles (6,378 kilometers).

### Exercise 4

Let's do Exercise 3 again, but correctly this time. What cities in the United States are actually closest to Morocco using Haversine distance?

In [ ]:
from sklearn.metrics import pairwise_distances
import numpy as np

# get the cities we want to compare
# to avoid PerformanceWarning, sort the index first
df_cities_indexed = df_cities.set_index(["city", "country"]).sort_index()
# get Casablanca and all US cities
cities = [("Casablanca", "Morocco")] + list(df_cities_indexed.loc[pd.IndexSlice[:, "United States of America"], :].index)
cities

[('Casablanca', 'Morocco'),
 ('Aberdeen', 'United States of America'),
 ('Aberdeen', 'United States of America'),
 ('Abilene', 'United States of America'),
 ('Aiken', 'United States of America'),
 ('Akron', 'United States of America'),
 ('Alamogordo', 'United States of America'),
 ('Albany', 'United States of America'),
 ('Albany', 'United States of America'),
 ('Albany', 'United States of America'),
 ('Albert Lea', 'United States of America'),
 ('Albuquerque', 'United States of America'),
 ('Alexandria', 'United States of America'),
 ('Alexandria', 'United States of America'),
 ('Alice', 'United States of America'),
 ('Allakaket', 'United States of America'),
 ('Allentown', 'United States of America'),
 ('Alliance', 'United States of America'),
 ('Alpena', 'United States of America'),
 ('Alpine', 'United States of America'),
 ('Alton', 'United States of America'),
 ('Altoona', 'United States of America'),
 ('Amarillo', 'United States of America'),
 ('Ambler', 'United States of America

In [83]:
# convert latitude and longitude to radians
radians = np.pi / 180 * df_cities_indexed.loc[cities, ["lat", "lng"]]
radians = radians.sort_index()
radians["distance_to_casablanca_km"] = pairwise_distances(
    radians[["lat", "lng"]],
    radians.loc[("Casablanca", "Morocco")].values.reshape(1, -1),
    metric="haversine"
) * 6378  # multiply by the radius of the Earth in km
closest_us_cities_to_casablanca = radians.sort_values("distance_to_casablanca_km").head(2).iloc[1]
closest_us_cities_to_casablanca

lat                             0.788295
lng                            -1.173601
distance_to_casablanca_km    5167.224816
Name: (Calais, United States of America), dtype: float64

In [84]:
# get the province/state of the closest city
df_cities_indexed.loc[closest_us_cities_to_casablanca.name]

,,city_ascii,lat,lng,pop,iso2,iso3,province
city,country,,,,,,,
Calais,United States of America,Calais,45.165989,-67.242392,1781.5,US,USA,Maine
